# Regressão múltipla
A regressão múltipla ocorre quando temos mais de uma variável independente. 

In [21]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from statsmodels.compat import lzip
from sklearn import feature_selection
import statsmodels.regression.linear_model as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

/home/55018335734/.conda/envs/datascience/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Dados de desempenho de um servidor web
Os dados para este trabalho foram coletados de um servidor web que hospeda um site. As observações são as médias das variáveis por minuto: 
- Duracao_media_ms: Duração média do processamento de um Request HTTP (em milissegundos);
- Perc_medio_CPU: Percentual médio de ocupação da CPU;
- Load_avg_minute: Carga total dos processadores;
- Requests_média: Quantidade média de requests processados. 

In [11]:
df = pd.read_csv('servidor.csv')
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
Duracao_media_ms    29 non-null int64
Perc_medio_CPU      29 non-null float64
Load_avg_minute     29 non-null float64
Requests_média      29 non-null int64
dtypes: float64(2), int64(2)
memory usage: 1008.0 bytes


,Duracao_media_ms,Perc_medio_CPU,Load_avg_minute,Requests_média
count,29.000000,29.000000,29.000000,29.000000
mean,811.586207,47.975050,3.927406,631.793103
std,672.159129,43.282833,3.379525,391.095657
min,140.000000,6.866574,0.498327,56.000000
25%,150.000000,10.581583,0.650421,230.000000
50%,520.000000,20.151097,2.333277,720.000000
75%,1500.000000,98.170972,7.317686,890.000000
max,1950.000000,121.235760,9.580188,1535.000000


In [14]:
results = smf.ols('Duracao_media_ms ~ Perc_medio_CPU + Load_avg_minute + Requests_média', data=df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Duracao_media_ms   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.755e+04
Date:                Fri, 10 Nov 2017   Prob (F-statistic):           1.17e-41
Time:                        14:11:46   Log-Likelihood:                -118.48
No. Observations:                  29   AIC:                             245.0
Df Residuals:                      25   BIC:                             250.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          20.7948      6.316      3.292      0.003       7.786      33.803
Perc_medio_CPU      0.2873      0.285      1.009      0.323      -0.299       0.874
Load_avg_minute   194.4347      3.828     50.799      0.000     186.552     202.318
Requests_média      0.0212      0.010      2.206      0.037       0.001       0.041
==============================================================================
Omnibus:                        0.521   Durbin-Watson:                   2.480
Prob(Omnibus):                  0.771   Jarque-Bera (JB):                0.611
Skew:                          -0.081   Prob(JB):                        0.737
Kurtosis:                       2.308   Cond. No.                     1.67e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.67e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
X = df.drop('Duracao_media_ms',axis=1)
Xe = sm.add_constant(X,prepend=True)
vif = [variance_inflation_factor(Xe.values, i) for i in range(Xe.shape[1])]
vif_s =pd.Series(vif, index =Xe.columns)
print(vif_s)

const               4.817213
Perc_medio_CPU     17.701874
Load_avg_minute    19.506190
Requests_média      1.645110
dtype: float64


In [29]:
name = ['Lagrange multiplier statistic', 'p-value', 
        'f-value', 'f p-value']
testeH = het_breuschpagan(results.resid, results.model.exog)
lzip(name, testeH)

[('Lagrange multiplier statistic', 3.6684353081331946),
 ('p-value', 0.29956464547347383),
 ('f-value', 1.2068063937739497),
 ('f p-value', 0.32771840796145069)]

# Avaliação do modelo
- Modelo: y = 0.2873x1 + 194.4347x2 + 0.0212x3 + 20.7948
- R quadrado ajustado: 0.999;
- Significância da regressão (F-test): Rejeitamos a hipótese nula de coeficientes zerados;
- Significância dos coeficientes: O Coeficiente "Perc_medio_CPU"  poderia ser retirado do modelo;
- Multicolinearidade (VIF): Temos multicolinearidade entre "Perc_medio_CPU" e "Load_avg_minute", porém, não há muito o que fazer sobre isso;
- Heterocedasticidade: De acordo com o teste de Breusch-Pagan (p-value > 0.05) o modelo é homocedástico. 